### Kraus objective builder function

In [2]:
using Symbolics
using LinearAlgebra
using DynamicPolynomials

Auxilary function for square Frobenius norm of the matrix $\left\| M \right\|_F^2$:

In [3]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

#### Consider Kraus evolution of quantum system

Consider Kraus map - a linear Completely Positive Trace Preserving (CP-TP) evolution of density matrix $\rho$:
\begin{aligned}
    \rho(t) & = \sum_{k=1}^{\ell} E_k^{} \rho(0) E_k^{\dagger}.
\end{aligned}

The operators $\{E_k \}_{k=1}^{\ell}$ are called Kraus operators and satisfy the completeness relation $\sum_{k=1}^{\ell}E_k^\dagger E_k = \mathbb{1}$ with $\ell \leq n^2-1$.

#### The corresponding minimization problem

 Terms for the objective for minimization for evolution from $\rho_{(0)}$ to $\rho_{(1)}$: 
\begin{align}
\left\| \sum_{k=1}^\ell \hat{E}_k \rho_{(0)} \hat{E}_k^{\dagger} - \rho_{(1)} \right\|_F^2
\rightarrow \operatorname{min}
\end{align} 
Under constraint:
\begin{align}
\sum_{k=1}^\ell \hat{E}_k^\dagger \hat{E}_k  = \mathbb{1}.
\end{align}

For the time series of $\rho(t)$:
\begin{align}
    \sum_{i=0}^{N-1} \left\| \sum_{k=1}^\ell \hat{E}_k \rho_{(i)} \hat{E}_k^{\dagger} - \rho_{(i+1)} \right\|_F^2 \rightarrow \operatorname{min}
\end{align}

under constrain:
\begin{align}
\left \| \sum_{k=1}^\ell \hat{E}_k^\dagger \hat{E}_k  -\mathbb{1} \right\|_F^2 = 0
\end{align}

The following function will essemble the Krauss objective from the time series of density matrix evolution:

In [62]:
import Base.real
function real(p::AbstractPolynomial)
    sum(real(coef) * mon for (coef, mon) in zip(coefficients(p), monomials(p)) if ~isapproxzero(abs(coef)))
end

real (generic function with 37 methods)

In [63]:
function kraus(ρ, K...) 
    obj = 0
    for i in 1:length(ρ)-1
        obj += frobenius_norm2(sum(k * ρ[i] * k' for k in K) - ρ[i+1])
    end
    constr = frobenius_norm2(sum(k' * k for k in K) - I)
    return real(obj), real(constr)
end

kraus (generic function with 1 method)

Consider toy example

In [6]:
@polyvar a1[1:2, 1:2]
@polyvar b1[1:2, 1:2]
K1 = 1.0 * a1 + im * b1

@polyvar a2[1:2, 1:2]
@polyvar b2[1:2, 1:2]
K2 = 1.0 * a2 + im * b2

ρ1 = [.5 -im
     +im  .5]
ρ2 = [.6 0
     0  .4]
ρ3 = [.7 0
     0  .3]
ρ4 = [.9 0
     0  .1]

ρ = [ρ1, ρ2, ρ3, ρ4]

4-element Vector{Matrix{ComplexF64}}:
 [0.5 + 0.0im 0.0 - 1.0im; 0.0 + 1.0im 0.5 + 0.0im]
 [0.6 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.4 + 0.0im]
 [0.7 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.3 + 0.0im]
 [0.9 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.1 + 0.0im]

In [7]:
#obj, constr = kraus(ρ, K1, K2)

In [8]:
#using TSSOS

#opt,sol,data = tssos_first(obj, variables(obj));

In [9]:
include("LiPoSID.jl")

Main.LiPoSID

In [10]:
using Random
seed = 1000
w = 0.0
t_list = [0:0.01:10;]
ρ = LiPoSID.rand_Linblad_w_noise(seed, w, t_list)

1001-element Vector{Matrix{ComplexF64}}:
 [0.2339834067667915 + 0.0im -0.2224241675173374 - 0.1814852900666712im; -0.2224241675173374 + 0.1814852900666712im 0.7660165932332086 + 0.0im]
 [0.24337078560225395 - 2.6587691635692076e-18im -0.21481344399763366 - 0.17275705665249125im; -0.21481344399763366 + 0.17275705665249125im 0.7566292143977461 + 6.915661697276398e-19im]
 [0.2518502320436706 - 6.1306172256878415e-18im -0.20701265214699277 - 0.163908846126603im; -0.20701265214699277 + 0.163908846126603im 0.7481497679563295 - 2.3906093849869434e-19im]
 [0.259443723137589 - 7.874039136793213e-18im -0.1991129042910502 - 0.1549679418520718im; -0.1991129042910502 + 0.15496794185207183im 0.740556276862411 - 1.8787068656436976e-19im]
 [0.26617886564301324 - 9.440831223488327e-18im -0.19119832967780218 - 0.1459617110588266im; -0.19119832967780218 + 0.1459617110588266im 0.7338211343569867 - 8.379007440964194e-19im]
 [0.2720883023123789 - 5.066084239701028e-18im -0.18334596502225053 - 0.136917368816

In [11]:
#=using Plots
bv = LiPoSID.bloch(ρ)
plot(bv')
=#

In [12]:
obj, constr = kraus(ρ, K1, K2)

(105.58520325468992a1₁₋₁⁴ - 179.42849395333548a1₁₋₁³a1₁₋₂ + 246.81906668664334a1₁₋₁³b1₁₋₂ + 211.17040650937983a1₁₋₁²a1₂₋₁² - 179.4284939533355a1₁₋₁²a1₂₋₁a1₂₋₂ + 246.81906668664334a1₁₋₁²a1₂₋₁b1₂₋₂ + 514.8158831923638a1₁₋₁²a1₁₋₂² - 246.81906668664334a1₁₋₁²a1₁₋₂b1₁₋₁ - 208.01186129499766a1₁₋₁²a1₁₋₂b1₁₋₂ + 114.26199004660405a1₁₋₁²a1₂₋₂² - 246.81906668664334a1₁₋₁²a1₂₋₂b1₂₋₁ + 211.17040650937983a1₁₋₁²b1₁₋₁² - 179.42849395333548a1₁₋₁²b1₁₋₁b1₁₋₂ + 211.17040650937983a1₁₋₁²b1₂₋₁² - 179.4284939533355a1₁₋₁²b1₂₋₁b1₂₋₂ + 590.266330775344a1₁₋₁²b1₁₋₂² + 114.26199004660405a1₁₋₁²b1₂₋₂² + 211.17040650937983a1₁₋₁²a2₁₋₁² - 179.42849395333548a1₁₋₁²a2₁₋₁a2₁₋₂ + 246.81906668664334a1₁₋₁²a2₁₋₁b2₁₋₂ + 438.27911693724906a1₁₋₁²a2₁₋₂² - 246.81906668664334a1₁₋₁²a2₁₋₂b2₁₋₁ + 211.17040650937983a1₁₋₁²b2₁₋₁² - 179.42849395333548a1₁₋₁²b2₁₋₁b2₁₋₂ + 438.27911693724906a1₁₋₁²b2₁₋₂² - 179.4284939533355a1₁₋₁a1₂₋₁²a1₁₋₂ + 246.81906668664334a1₁₋₁a1₂₋₁²b1₁₋₂ + 801.1077862915173a1₁₋₁a1₂₋₁a1₁₋₂a1₂₋₂ - 208.01186129499766a1₁₋₁a1₂₋₁a1

In [13]:
#obj_min, minmizer = LiPoSID.minimize_global(obj)

In [14]:
#solution = LiPoSID.minimize_local(obj, rand(length(variables(obj))))

In [15]:
exact_solution = variables(obj) => rand(length(variables(obj)))

PolyVar{true}[a1₁₋₁, a1₂₋₁, a1₁₋₂, a1₂₋₂, b1₁₋₁, b1₂₋₁, b1₁₋₂, b1₂₋₂, a2₁₋₁, a2₂₋₁, a2₁₋₂, a2₂₋₂, b2₁₋₁, b2₂₋₁, b2₁₋₂, b2₂₋₂] => [0.7154620008460185, 0.30186397778513974, 0.20479394978338905, 0.12604726169119185, 0.4905246835000866, 0.2712681967054573, 0.09231168172075843, 0.6323211557874976, 0.901469780610935, 0.8481633659495147, 0.5647240083236703, 0.6692331978233357, 0.2331997452703296, 0.5337011064156874, 0.5154938307695669, 0.7565546089038714]

In [23]:
K1_exact = convert(Matrix{ComplexF64}, subs(K1, exact_solution))
K2_exact = convert(Matrix{ComplexF64}, subs(K2, exact_solution))

2×2 Matrix{ComplexF64}:
  0.90147+0.2332im    0.564724+0.515494im
 0.848163+0.533701im  0.669233+0.756555im

In [ ]:
function timeevolution_kraus(t_span, ρ₀, K_list)
    ρ = [ρ₀]
    for t = 1:length(t_span)
        push!(ρ, Hermitian(sum([K[i]* ρ[end] * K[i]' for i = 1:length(K)])
    end   
end  
        
tₘₐₓ = 2.0 # maximum time 
Δt = 0.05     # time step
time_span = [0:Δt:tₘₐₓ;] # time span

ρ₀ = rand_dm(2)       

K1 = rand(2,2) + im*rand(2,2)
K2 = rand(2,2) + im*rand(2,2)

K_list = [K1, K2]
        
ρ = timeevolution.kraus(t_span, ρ₀, K_list)
        using Plots
plot(LiPoSID.bloch(ρ)')

In [49]:
ρ = [LiPoSID.rand_dm(2)]

for k =1:10
    push!(ρ, Hermitian(K1_exact * ρ[end] * K1_exact' + K2_exact * ρ[end] * K2_exact')) 
end

In [64]:
obj, constr = kraus(ρ, K1, K2);

In [65]:
subs(obj, exact_solution)

0.0

In [86]:
solution = LiPoSID.minimize_local(obj, 2 .+ rand(length(variables(obj))))
subs(obj, solution)

0.9432373046875

In [88]:
using TSSOS

opt,sol,data = tssos_first(obj, variables(obj));

************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[73, 32, 8]
[1, 2, 2]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 73.
Assembling the SDP...
There are 2029 affine constraints.
Solving the SDP...
MOSEK warning 52: A numerically large lower bound value  4.2e+11 is specified for constraint '' (0).
MOSEK warning 53: A numerically large upper bound value  4.2e+11 is specified for constraint '' (0).
MOSEK warning 52: A numerically large lower bound value -7.1e+10 is specified for constraint '' (1).
MOSEK warning 53: A numerically large upper bound value -7.1e+10 is specified for constraint '' (1).
MOSEK warning 52: A numerically large lower bound value  9.1e+09 is specified for constraint '' (2).
MOSEK warning 53: A numerically large upper bound value  9.1e+09 is specified f

In [93]:
a1 * a2

2×2 Matrix{Polynomial{true, Int64}}:
 a1₁₋₁a2₁₋₁ + a1₁₋₂a2₂₋₁  a1₁₋₁a2₁₋₂ + a1₁₋₂a2₂₋₂
 a1₂₋₁a2₁₋₁ + a1₂₋₂a2₂₋₁  a1₂₋₁a2₁₋₂ + a1₂₋₂a2₂₋₂